# How to get and use MOC data from VESPA in Python with PyVO

Illustrate search for MOC data in VESPA using PyVO.
It also gives an example of a simple query to group together MOCs with the same value for a specific field (channel id), and then give the elements that are in the area defined by the intersection of all of those MOCs.

## Import required modules

In [1]:
import pyvo
from mpl_toolkits.basemap import Basemap

## Search a table in the RegTAP registry

In this example we are looking for services with data about Venus since we know that at least one of them has MOC (Multi-Order Coverage map) data, this service is "VVEx - VIRTIS/Venus Express database".

### Search the database containg a table that has the following keyword in its description

In [2]:
keyword = "Venus"

We use pyvo's search function to search for EPN-TAP services, since we want a "coverage" field for the MOCs.

In [7]:
databasesList = pyvo.registry.search(datamodel="epntap", servicetype='tap',keywords=keyword, includeaux=True)

In [8]:
for d in databasesList :
    print(d.short_name)


Cassini RPWS
BDIP
HST Planeto
VVEx
MESS-MAG-VSO
MESS-MAG-VSO-AVG
PVO-PHEM-ASC
PVO-PHEM-BIN
VEX-ASPERA4-DER
VEX-ASPERA4-ENG
VEX-ASPERA4-RAW
VEX-HIGH-MAG


### Search the tables in the databases that were found

First, we need to get the access url of all the databases found in the previous section.
So we get the list of values in the field access_url of databasesList.
But this list contains duplicates, so we create a set from this list to have a set of unique urls (since a set doesn't have duplicates).

In [5]:
urlsList = set(databasesList.to_table().field("access_urls"))

For each url in the previous list we create a TAPService object representing the TAP service corresponding to this url.
We can now search in this service all the tables whose name or description match with the keyword.

We add the results in an array of dictionnaries that contains both the access url and the name of the table.

In [6]:
tables = []

for url in urlsList :
    print("Searching in service " + url)
    service = pyvo.dal.TAPService(url)
    searchResult = service.search("SELECT table_name FROM tap_schema.tables WHERE table_name LIKE '%"+keyword+"%' OR description LIKE '%"+keyword+"%'")
    for tableFound in searchResult["table_name"] :
        tables.append({"url":url,"name":tableFound})
print("DONE")

Searching in service https://vo-pds-ppi.igpp.ucla.edu/tap
Searching in service http://voparis-tap-maser.obspm.fr/tap
Searching in service http://voparis-tap-planeto.obspm.fr/tap
Searching in service http://vo.lmd.jussieu.fr/tap
DONE


In [7]:
for table in tables :
    print(table["name"]," at ", table["url"])

vex_cleaned_high_res_mag.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_raw.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
mess_mag_calibrated_vso.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_derived.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
mess_mag_calibrated_vso_avg.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
pvo_ephem_vso_asc.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_eng.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
pvo_ephem_vso_bin.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vvex.epn_core  at  http://voparis-tap-planeto.obspm.fr/tap


## Only keep tables with a 'coverage' key (and so maybe a MOC)

In this section, we want to only keep tables that may contain MOC data, and put them in the array tablesWithMOC.
So, for each table, we use its access url to create a TAPService object, as we have done before.
And then, we use the following ADQL query to check if there are rows with a coverage.

If there is no coverage field (an Exception is thrown and catched) or if this field is null, then it returns false (there is no MOC data), otherwise it returns true.

In [8]:
def doContainMOC(table) :
    service = pyvo.dal.TAPService(table["url"])
    try :
        searchResult = service.search("SELECT TOP 1 * FROM "+table["name"]+" WHERE coverage IS NOT NULL")
        if searchResult == "" :
            return False
        else :
            return True
    except :
        return False

In [9]:
tablesWithMOC = []
for table in tables :
    print("Checking table ",table["name"])
    if doContainMOC(table) :
        tablesWithMOC.append(table)

Checking table  vex_cleaned_high_res_mag.epn_core
Checking table  vex_aspera4_els_raw.epn_core
Checking table  mess_mag_calibrated_vso.epn_core
Checking table  vex_aspera4_els_derived.epn_core
Checking table  mess_mag_calibrated_vso_avg.epn_core
Checking table  pvo_ephem_vso_asc.epn_core
Checking table  vex_aspera4_els_eng.epn_core
Checking table  pvo_ephem_vso_bin.epn_core
Checking table  vvex.epn_core


In [10]:
for table in tablesWithMOC :
    print(table["name"])

vvex.epn_core


## Display the first 10 elements of a table
Here, and in the rest of this notebook, we only consider the first table found in the previous section

In [11]:
table = tablesWithMOC[0]
tapService = pyvo.dal.TAPService(table["url"])

data = tapService.search("SELECT TOP 10 * FROM " + table["name"]).to_table()
print(data)

granule_uid granule_gid   obs_id  ... science_case_id sc_pointing_mode
                                  ...                                 
----------- ----------- --------- ... --------------- ----------------
 VV1518_02C  calibrated VV1518_02 ...               0             NULL
 VV1518_02G    geometry VV1518_02 ...               0             NULL
 VV1518_08C  calibrated VV1518_08 ...               2           MOSAIC
 VV1518_08G    geometry VV1518_08 ...               2           MOSAIC
 VV1518_07C  calibrated VV1518_07 ...               2           MOSAIC
 VV1518_07G    geometry VV1518_07 ...               2           MOSAIC
 VV1518_11C  calibrated VV1518_11 ...               1        NADIR_POW
 VV1518_11G    geometry VV1518_11 ...               1        NADIR_POW
 VV1518_09C  calibrated VV1518_09 ...               2           MOSAIC
 VV1518_09G    geometry VV1518_09 ...               2           MOSAIC


## Import modules for Aladin and MOCs

In [12]:
from mocpy import MOC
from ipyaladin import Aladin

## How to load a MOC in Aladin

In this section we will see how to load a MOC in Aladin.

### Get the data

In [13]:
table = tablesWithMOC[0]
tapService = pyvo.dal.TAPService(table["url"])

data = tapService.search("SELECT TOP 5 * FROM " + table["name"] + " WHERE coverage IS NOT NULL").to_table()
print(data)

granule_uid granule_gid   obs_id  ... science_case_id sc_pointing_mode
                                  ...                                 
----------- ----------- --------- ... --------------- ----------------
 VV1518_02C  calibrated VV1518_02 ...               0             NULL
 VV1518_02G    geometry VV1518_02 ...               0             NULL
 VV1518_08C  calibrated VV1518_08 ...               2           MOSAIC
 VV1518_08G    geometry VV1518_08 ...               2           MOSAIC
 VV1518_07C  calibrated VV1518_07 ...               2           MOSAIC


### Load Aladin and add the data that was fetched just above

Here, and in the following examples, we are using the HiPS (Hierarchical Progressive Surveys) of Venus.
Other HiPS can be found here http://voparis-srv-paris.obspm.fr/vo/planeto/hips/.



In [14]:
aladin = Aladin(
    coo_frame="body",
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Venus_Magellan_C3-MDIR-2025m/"
)
aladin

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…

### Load the MOCs

First, for each MOC, we create a MOC object with mocpy and then we convert it to a dictionnary, since it's the format that ipyaladin expects in the function add_moc_from_dict.
And we change the FOV so that the view is refreshed.

In [15]:
for item in data :
    mocObject = MOC.from_str(item["coverage"])
    jsonMoc = mocObject.serialize(format='json', optional_kw_dict=None, pre_v2=False)
    aladin.add_moc_from_dict(jsonMoc, {'opacity' : 0.5, 'name' : item["granule_uid"]})
aladin.fov = aladin.fov+1.0

We center the view on one of the MOC

In [16]:
mocCenter = MOC.from_str(data[0]["coverage"]).barycenter()
aladin.target = mocCenter.to_string()

## Union and intersection of MOCs

In this section, we are going to see how to search specific MOCs.
Here we only consider MOCs in the Northern Hemisphere, whose dataproduct_type is the spectral cube and whose observation minimim local time was before 20 P.M.

Then, we will group those MOCs in three categories, corresponding to the channel_id field associated to them.
And finally, we will search all the elements whose MOC is in a MOC defined as the intersection of those three groups.

### Get the data

In [17]:
table = tablesWithMOC[0]
tapService = pyvo.dal.TAPService(table["url"])
query = 'SELECT * FROM '+table["name"]+' WHERE c1min >= 0.0 AND c1min <= 360.0 AND c1max >= 0.0 AND c1max <= 360.0 AND c2min >= 0.0 AND c2min <= 90.0 AND c2max >= 0.0 AND c2max <= 90.0 AND dataproduct_type LIKE \'%sc%\' AND local_time_min <= 20'

data = tapService.search(query).to_table()

In [18]:
print("Number of results: ", len(data))
print("granule_uid\tproduct\tc1min ; c1max\t\tc2min ; c2max\t\tlocal_time_min")
for e in data :
    print(e["granule_uid"],"\t",e["dataproduct_type"], "\t", e["c1min"], ";", e["c1max"], "\t", e["c2min"], ";", e["c2max"],"\t", e["local_time_min"])

Number of results:  16
granule_uid	product	c1min ; c1max		c2min ; c2max		local_time_min
VV0041_02C 	 sc 	 356.868 ; 357.605 	 7.18175 ; 69.9193 	 2.98
VV0047_00C 	 sc 	 187.423 ; 3.6804 	 4.82867 ; 89.6728 	 3.8
VV0058_00C 	 sc 	 203.97 ; 207.854 	 4.6971 ; 84.4626 	 16.44
VV0062_00C 	 sc 	 205.277 ; 209.194 	 49.8892 ; 79.8392 	 17.17
VV0060_05C 	 sc 	 206.929 ; 210.947 	 4.31105 ; 84.6661 	 16.64
VT0033_00C 	 sc 	 165.763 ; 345.301 	 6.09572 ; 89.6878 	 2.16
VT0060_01C 	 sc 	 207.105 ; 209.219 	 17.0944 ; 56.5911 	 16.76
VT0062_00C 	 sc 	 205.638 ; 209.208 	 54.6843 ; 77.9203 	 17.17
VT0041_03C 	 sc 	 357.249 ; 357.394 	 17.0473 ; 50.3341 	 2.99
VT0047_00C 	 sc 	 187.808 ; 189.872 	 9.29254 ; 57.4396 	 15.39
VT0058_00C 	 sc 	 204.138 ; 206.249 	 16.853 ; 56.5184 	 16.55
VI0041_02C 	 sc 	 356.868 ; 357.605 	 7.18498 ; 69.9257 	 2.98
VI0047_00C 	 sc 	 187.422 ; 3.6675 	 4.79193 ; 89.6692 	 3.8
VI0058_00C 	 sc 	 203.961 ; 207.635 	 4.52508 ; 84.1857 	 16.45
VI0060_05C 	 sc 	 206.929 ; 2

### Group the MOCs by channel_id

Here we are using a dictionnary: each key is a channel_id value, the value associated to it is an array containing all the MOCs whose channel_id is equal to this key.
In the following block of code we are filling this dictionnary.

In [19]:
mocGroups = {"VIRTIS_M_VIS":[], "VIRTIS_M_IR":[], "VIRTIS_H":[]}
for element in data :
    mocGroups[element["channel_id"]].append(MOC.from_str(element["coverage"]))

For each channel_id we are now using mocpy to make an union of all the MOCs in the array corresponding to this channel_id.
So we add the first element to mocUnion, we remove it from the array, then for each element of the array we make an union between mocUnion and this element, and finally, we replace the array of MOCs with the MOC mocUnion at the corresponding key.

So, the following block of code converts a dictionnary of MOCs array to a dictionnary of MOCs that covers the same area as the array of MOCs that was previously at the same key.

In [20]:
for key in list(mocGroups.keys()) :
    if(len(mocGroups[key]) > 0):
        mocUnion = MOC.new_empty(29)
        for moc in mocGroups[key] :
            mocUnion = mocUnion.union(moc)
        mocGroups[key] = mocUnion
    else :
        mocGroups.pop(key)

In [21]:
aladin2 = Aladin(
    coo_frame='body',
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Venus_Magellan_C3-MDIR-2025m/"
)
aladin2

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…

In [22]:
for group, moc in mocGroups.items() :
    jsonMoc = moc.serialize(format='json', optional_kw_dict=None, pre_v2=False)
    aladin2.add_moc_from_dict(jsonMoc, {'opacity' : 0.5, 'name' : group})
aladin2.fov = aladin2.fov+1.0

In [23]:
mocCenter = list(mocGroups.values())[0].barycenter()
aladin2.target = mocCenter.to_string()

### Intersection of MOCs

Here, we get the intersection of the three MOCs in mocGroups

In [24]:
mocList = []
for moc in mocGroups.values() :
    mocList.append(moc)
    
if(len(mocGroups) > 0):
    mocIntersection = mocList[0]
    mocList.pop(0)
    for moc in mocList :
        mocIntersection = mocIntersection.intersection(moc)

We can plot this intersection in Aladin if needed:

In [25]:
jsonMoc = mocIntersection.serialize(format='json', optional_kw_dict=None, pre_v2=False)
aladin2.add_moc_from_dict(jsonMoc, {'opacity' : 0.5, 'name' : 'intersection'})
aladin2.fov = aladin2.fov+1.0

In [26]:
mocCenter = mocIntersection.barycenter()
aladin2.target = mocCenter.to_string()

Then, for each item in the data that we got in the "Get the data" section, we can get the intersection between the MOC associated to this item and the MOC corresponding to the intersection of the three groups of MOCs (mocIntersection).

If this intersection is not empty, then this item is in mocIntersection.
So we get a list of items whose MOC intersects with the MOC of other items with the two other types of channel_id

In [27]:
results = []
for item in data :
    moc = MOC.from_str(item["coverage"])
    if not moc.intersection(mocIntersection).empty() :
        results.append(item)        

In [28]:
print(len(results), " results found:")
for item in results :
    print(item["granule_uid"])

15  results found:
VV0041_02C
VV0047_00C
VV0058_00C
VV0062_00C
VV0060_05C
VT0060_01C
VT0062_00C
VT0041_03C
VT0047_00C
VT0058_00C
VI0041_02C
VI0047_00C
VI0058_00C
VI0060_05C
VI0062_00C


## Select an area in Aladin and search elements within

### Import required modules

In [29]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from ipywidgets import Layout, Box, widgets

### Draw a MOC with the mouse cursor

When the user click somewhere in Aladin (on the surface), the position of the click is saved and we update the ranges of c1 and c2 (longitude and latitude), that will be used for the query to avoid having to much data.

To get the moc of the selected area, click on the button 'Add selection' and you will see it in Aladin (you might need to rotate the sphere to update the view).

But before doing so, we need to add an event to the button and to the Aladin box. This will be added in the following blocks of code.

In [30]:
aladin3 = Aladin(
    layout=Layout(width="70%"),
    coo_frame='body',
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Venus_Magellan_C3-MDIR-2025m/"
)

button = widgets.Button(description="Add selection")

box_layout = Layout(
    display="flex", flex_flow="row", align_items="stretch", width="100%"
)
box = Box(children=[aladin3, button], layout=box_layout)
box

Box(children=(Aladin(coo_frame='body', layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame',…

In [31]:
def updateBounds(areaPoints, ra, dec) :
    if areaPoints["c1min"] is None or areaPoints["c1min"] > ra :
        areaPoints["c1min"] = ra
    if areaPoints["c1max"] is None or areaPoints["c1max"] < ra :
        areaPoints["c1max"] = ra
    if areaPoints["c2min"] is None or areaPoints["c2min"] > dec :
        areaPoints["c2min"] = dec
    if areaPoints["c2max"] is None or areaPoints["c2max"] < dec :
        areaPoints["c2max"] = dec

In [32]:
def addMocSelectionToAladin(*_) :
    if len(areaPoints) > 1 :
        polygonCoordinates = SkyCoord(areaPoints["ra"]*u.deg, areaPoints["dec"]*u.deg)
        mocSelection = MOC.from_polygon_skycoord(polygonCoordinates, max_depth=15)
        aladin3.add_moc_from_dict(mocSelection.serialize("json"), {"color": "red", "opacity": 0.5, "name": "selection"})
        areaPoints["moc"] = mocSelection
aladin3.fov = aladin3.fov+1.0

In [33]:
areaPoints =  {"moc" : None, "c1min": None, "c1max" : None, "c2min" : None, "c2max" : None, "ra" : [], "dec" : []}

def addToAreaPoints(data) :
    updateBounds(areaPoints, data["ra"],data["dec"])
    areaPoints["ra"].append(data["ra"])
    areaPoints["dec"].append(data["dec"])

And we can now add the events:

In [34]:
aladin3.add_listener("click", addToAreaPoints)
button.on_click(addMocSelectionToAladin)

### Get the data

We get all the data that may intersect with the searched area, but we limit it at 5000 results here so that it's not to slow.

In [35]:
tapService = pyvo.dal.TAPService(table["url"])
query = 'SELECT TOP 5000 * FROM '+table["name"]+' WHERE ((c1min BETWEEN '+str(areaPoints["c1min"])+' AND '+str(areaPoints["c1max"])+') OR (c1max BETWEEN '+str(areaPoints["c1min"])+' AND '+str(areaPoints["c1max"])+')) AND ((c2min BETWEEN '+str(areaPoints["c2min"])+' AND '+str(areaPoints["c2max"])+') OR (c2max BETWEEN '+str(areaPoints["c2min"])+' AND '+str(areaPoints["c2max"])+'))'

data = tapService.search(query).to_table()

print("Number of results: ", len(data))

Number of results:  5000


### Search MOCs that intersect with it and get the associated elements

Then, for each item in the data that we got in the "Get the data" section, we can get the intersection between the MOC associated to this item and the MOC corresponding to the searched area.

If this intersection is not empty, then this item is in the searched area.

In [36]:
results = []
for item in data :
    if areaPoints["moc"] is None :
        print("The search area is empty. Please select an area and click on the button")
    else : 
        if item["coverage"] is not None and item["coverage"] != "" :
            moc = MOC.from_str(item["coverage"])
            if not moc.intersection(areaPoints["moc"]).empty() :
                results.append(item)

In [37]:
print(len(results), " results found:")
for item in results :
    print(item["granule_uid"])

280  results found:
VV1748_02C
VV1748_02G
VV1746_02C
VV1746_02G
VV1719_00C
VV1719_00G
VV1719_01C
VV1719_01G
VV1654_09C
VV1654_09G
VV1652_08C
VV1652_08G
VV1650_08C
VV1650_08G
VV1647_05C
VV1647_05G
VV1623_05C
VV1623_05G
VV1621_04C
VV1621_04G
VV1618_05C
VV1618_05G
VV1616_05C
VV1616_05G
VV1653_08C
VV1653_08G
VV1240_00C
VV1240_00G
VV1617_05C
VV1617_05G
VV0654_03C
VV0654_03G
VV0656_03C
VV0656_03G
VV0628_09C
VV0628_09G
VV0377_11C
VV0377_11G
VV0390_09C
VV0390_09G
VV0390_13C
VV0390_13G
VV1992_06C
VV1992_06G
VV2081_04C
VV2081_04G
VV2081_07C
VV2081_07G
VV2081_05C
VV2081_05G
VV2079_06C
VV2079_06G
VV2079_05C
VV2079_05G
VV2079_07C
VV2079_07G
VV2078_04C
VV2078_04G
VV2078_05C
VV2078_05G
VV2077_05C
VV2077_05G
VV2075_05C
VV2075_05G
VV2075_04C
VV2075_04G
VV2074_04C
VV2074_04G
VV2072_04C
VV2072_04G
VV2071_04C
VV2071_04G
VV2120_01C
VV2120_01G
VV2118_01C
VV2118_01G
VV2117_00C
VV2117_00G
VV2117_01C
VV2117_01G
VV2116_02C
VV2116_02G
VV2081_06C
VV2081_06G
VV2077_04C
VV2077_04G
VV2074_05C
VV2074_05G
VV1846_03C
V

### Load the MOCs of some of the results in Aladin

In [38]:
for i in range(min(10, len(results))) :
    moc = MOC.from_str(results[i]["coverage"])
    aladin3.add_moc_from_dict(moc.serialize("json"), {"opacity": 0.5, "name": results[i]["granule_uid"]})
aladin3.fov = aladin3.fov+1.0